# Import Library

In [1]:
import gmaps
import pandas as pd
import numpy as np
import requests
import json
from config import gkey
# Configure gmaps
gmaps.configure(api_key=gkey)

# Read csv file

In [2]:
# Create a dataframe
weather_df = pd.read_csv('../WeatherPy/weatherpy_data.csv')
weather_df.head()

,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,arraial do cabo,br,2020-03-14 21:52:15,-22.97,-42.03,78.28,88.0,16.0,12.30
1,koumac,nc,2020-03-14 21:56:27,-20.57,164.28,78.42,87.0,100.0,30.24
2,yellowknife,ca,2020-03-14 21:54:15,62.46,-114.35,10.40,66.0,90.0,8.05
3,mehamn,no,2020-03-14 21:49:31,71.04,27.85,15.51,91.0,100.0,13.96
4,vao,nc,2020-03-14 21:56:18,-22.67,167.48,83.75,82.0,99.0,29.95


## Create a heat map that displays the humidity for every city 

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]
Humidity = weather_df["Humidity (%)"].astype(float)
maxhumidity = Humidity.max()

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity,
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Find ideal weather condition

### Narrow down the DataFrame to find an ideal weather condition.

* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.

In [5]:
new_weather_df = weather_df.loc[(weather_df["Max Temperature (F)"] > 70) & 
                                (weather_df["Max Temperature (F)"] < 80) & 
                                (weather_df["Wind Speed (mph)"]<10) &
                                (weather_df["Cloudiness (%)"] == 0) , :]

new_weather_df.reset_index(inplace=True)
del new_weather_df['index']
print(f"There are {len(new_weather_df)} cities satisfying the ideal weather condition.")
new_weather_df

There are 9 cities satisfying the ideal weather condition.


,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,werda,bw,2020-03-14 21:56:29,-25.27,23.28,71.15,40.0,0.0,6.35
1,bataipora,br,2020-03-14 21:56:36,-22.30,-53.27,71.31,72.0,0.0,2.42
2,bachaquero,ve,2020-03-14 21:56:44,9.97,-71.15,76.08,76.0,0.0,0.83
3,tecoanapa,mx,2020-03-14 21:56:49,16.52,-98.75,75.36,78.0,0.0,1.21
4,el real de santa maria,pa,2020-03-14 21:57:01,8.11,-77.73,70.45,92.0,0.0,2.30
5,gorom-gorom,bf,2020-03-14 21:57:07,14.44,-0.24,75.27,11.0,0.0,9.42
6,angul,in,2020-03-14 21:57:09,20.85,85.10,78.42,53.0,0.0,5.32
7,swan hill,au,2020-03-14 21:57:44,-35.34,143.55,72.00,43.0,0.0,1.99
8,camapua,br,2020-03-14 21:50:26,-19.53,-54.04,73.67,83.0,0.0,7.25


## Build a Hotel Map

In [6]:
hotels = []

for i in range(len(new_weather_df)):
    lat = new_weather_df.loc[i]['Latitude']
    lng = new_weather_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": gkey
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    try:
        print(f"Processing Record {i}, City Name: {new_weather_df.loc[i]['City']}.")
        hotels.append(response['results'][0]['name'])
        
    except:
        hotels.append(np.nan)
        print("Oops, Nearest hotel not found!")
        
print("-----------------------------")        
print("Data Retrieval Complete")
print("-----------------------------")

Processing Record 0, City Name: werda.
Processing Record 1, City Name: bataipora.
Processing Record 2, City Name: bachaquero.
Processing Record 3, City Name: tecoanapa.
Processing Record 4, City Name: el real de santa maria.
Processing Record 5, City Name: gorom-gorom.
Processing Record 6, City Name: angul.
Processing Record 7, City Name: swan hill.
Processing Record 8, City Name: camapua.
-----------------------------
Data Retrieval Complete
-----------------------------


In [7]:
weather_data=new_weather_df.copy()
weather_data["Hotel Name"] = hotels
weather_data=weather_data.dropna()
weather_data

,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,werda,bw,2020-03-14 21:56:29,-25.27,23.28,71.15,40.0,0.0,6.35,Werda
1,bataipora,br,2020-03-14 21:56:36,-22.30,-53.27,71.31,72.0,0.0,2.42,Centro
2,bachaquero,ve,2020-03-14 21:56:44,9.97,-71.15,76.08,76.0,0.0,0.83,Bachaquero
3,tecoanapa,mx,2020-03-14 21:56:49,16.52,-98.75,75.36,78.0,0.0,1.21,Barra de Tecoanapa
4,el real de santa maria,pa,2020-03-14 21:57:01,8.11,-77.73,70.45,92.0,0.0,2.30,El Real
5,gorom-gorom,bf,2020-03-14 21:57:07,14.44,-0.24,75.27,11.0,0.0,9.42,Gorom-Gorom
6,angul,in,2020-03-14 21:57:09,20.85,85.10,78.42,53.0,0.0,5.32,Angul
7,swan hill,au,2020-03-14 21:57:44,-35.34,143.55,72.00,43.0,0.0,1.99,Swan Hill
8,camapua,br,2020-03-14 21:50:26,-19.53,-54.04,73.67,83.0,0.0,7.25,Camapuã


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in weather_data.iterrows()]
locations = weather_data[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))